In [ ]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans, AgglomerativeClustering
from sklearn.metrics import silhouette_score

PATH = "cad_data.csv"

df = pd.read_csv(PATH)
raw = df.copy()

for col in ["DISPATCH CREATED DATE", "ALARM DATE TIME", "CALL COMPLETE"]:
    df[col] = pd.to_datetime(df[col], errors="coerce")

df.insert(0, "CASE_ID", np.arange(1, len(df)+1))

def count_units(s):
    if pd.isna(s) or str(s).strip() == "":
        return 0
    parts = [p.strip() for p in str(s).split(",") if p.strip()]
    return len(parts)

df["UNITS_COUNT"] = raw["DISPATCH UNIT"].apply(count_units)

def hour_to_shift(h):
    if pd.isna(h):
        return np.nan
    if 0 <= h <= 7:
        return "A"
    if 8 <= h <= 15:
        return "B"
    return "C"

df["ALARM_HOUR"] = df["ALARM DATE TIME"].dt.hour
df["SHIFT_IMPUTED"] = df["ALARM_HOUR"].apply(hour_to_shift)
df["SHIFT_CLEAN"]   = df["SHIFT"].fillna(df["SHIFT_IMPUTED"])

df["RESOLVE_MIN"] = (df["CALL COMPLETE"] - df["ALARM DATE TIME"]).dt.total_seconds()/60
df["END_TO_CREATE_MIN"] = (df["CALL COMPLETE"] - df["DISPATCH CREATED DATE"]).dt.total_seconds()/60

df["DOW"]  = df["ALARM DATE TIME"].dt.day_name()
df["HOUR"] = df["ALARM DATE TIME"].dt.hour


In [ ]:
a. Data Issues

In [ ]:

# a.1 Range of data
date_min = df[["DISPATCH CREATED DATE","ALARM DATE TIME","CALL COMPLETE"]].min().min()
date_max = df[["DISPATCH CREATED DATE","ALARM DATE TIME","CALL COMPLETE"]].max().max()
print("a.1) Earliest:", date_min, " Latest:", date_max)

# a.2 Missing % per column
missing_pct = raw.isna().mean().mul(100).round(2)
print("\na.2) % Missing per column:\n", missing_pct.to_string())

# a.3 Notes on issues
print("""
a.3) Issues found:
- Time columns were strings -> converted to datetime with coercion.
- DISPATCH UNIT contains comma-separated lists with inconsistent spacing -> standardized and counted as UNITS_COUNT.
- SHIFT missing (~3%) -> imputed from ALARM_HOUR (A=0–7, B=8–15, C=16–23) into SHIFT_CLEAN.
- Some rows lack ALARM or CALL COMPLETE timestamps -> excluded only for metrics requiring durations.
- Added sequential CASE_ID for analysis stability.
""")

# a.4 Strategy already applied above. Save cleaned file for reproducibility.
out_path = "/mnt/data/cad_cleaned_analysis_ready.csv"
df.to_csv(out_path, index=False)
print("a.4) Cleaned dataset saved to:", out_path)


In [ ]:
b. Exploratory Data Analysis

In [ ]:

# b.1 Average resolve time (CALL COMPLETE - ALARM)
resolve_valid = df.dropna(subset=["RESOLVE_MIN"])
avg_resolve_min = resolve_valid["RESOLVE_MIN"].mean()
print(f"b.1) Average resolve time: {avg_resolve_min:.2f} minutes (n={len(resolve_valid)})")

# b.2 Average number of units dispatched
avg_units = df["UNITS_COUNT"].mean()
print(f"b.2) Average units dispatched: {avg_units:.2f}")

# b.3 Busiest shift (by number of incidents)
shift_counts = df["SHIFT_CLEAN"].value_counts()
busiest = shift_counts.idxmax()
print("b.3) Busiest shift:", busiest)
print(shift_counts)

# b.4 Hour x Day matrix with totals
pivot = pd.pivot_table(
    df.dropna(subset=["DOW","HOUR"]),
    index="HOUR", columns="DOW", values="CASE_ID", aggfunc="count", fill_value=0
).sort_index()

dow_order = ["Monday","Tuesday","Wednesday","Thursday","Friday","Saturday","Sunday"]
pivot = pivot.reindex(columns=dow_order)

pivot["TOTAL"] = pivot.sum(axis=1)
totals_row = pivot.sum(axis=0).to_frame().T
totals_row.index = ["TOTAL"]
matrix_with_totals = pd.concat([pivot, totals_row])
matrix_with_totals


In [ ]:
plt.figure(figsize=(10,5))
plt.plot(pivot.index, pivot["TOTAL"], marker="o")
plt.title("Total Alarms by Hour of Day")
plt.xlabel("Hour")
plt.ylabel("Count of Alarms")
plt.tight_layout()
plt.show()


In [ ]:
c. unsupervised learning

In [ ]:

# Feature engineering for clustering
model_df = df.dropna(subset=["RESOLVE_MIN","HOUR"]).copy()
model_df["HOUR_SIN"] = np.sin(2*np.pi*model_df["HOUR"]/24)
model_df["HOUR_COS"] = np.cos(2*np.pi*model_df["HOUR"]/24)

X = model_df[["UNITS_COUNT","RESOLVE_MIN","HOUR_SIN","HOUR_COS"]].values

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Method 1: KMeans
kmeans = KMeans(n_clusters=3, random_state=42, n_init=10)
k_labels = kmeans.fit_predict(X_scaled)
k_sil = silhouette_score(X_scaled, k_labels)

# Method 2: Agglomerative
agg = AgglomerativeClustering(n_clusters=3, linkage="ward")
a_labels = agg.fit_predict(X_scaled)
a_sil = silhouette_score(X_scaled, a_labels)

print(f"c.1) Silhouette — KMeans: {k_sil:.3f}, Agglomerative: {a_sil:.3f}")
best = "KMeans" if k_sil >= a_sil else "Agglomerative"
print("Best method:", best)

best_labels = k_labels if best=="KMeans" else a_labels
model_df["CLUSTER"] = best_labels

cluster_summary = model_df.groupby("CLUSTER").agg(
    UNITS_MEAN=("UNITS_COUNT","mean"),
    RESOLVE_MIN_MEAN=("RESOLVE_MIN","mean"),
    HOUR_MEDIAN=("HOUR","median"),
    COUNT=("CASE_ID","count")
).round(2).sort_index()

cluster_summary


In [ ]:

# Scatter: Units vs Resolve Time colored by cluster
plt.figure(figsize=(8,6))
plt.scatter(model_df["UNITS_COUNT"], model_df["RESOLVE_MIN"], c=model_df["CLUSTER"], s=18)
plt.xlabel("Units Dispatched")
plt.ylabel("Resolve Time (min)")
plt.title("Clusters: Units vs Resolve Time")
plt.tight_layout()
plt.show()


In [ ]:

# Scatter: Hour vs Resolve Time colored by cluster
plt.figure(figsize=(8,6))
plt.scatter(model_df["HOUR"], model_df["RESOLVE_MIN"], c=model_df["CLUSTER"], s=18)
plt.xlabel("Hour of Day")
plt.ylabel("Resolve Time (min)")
plt.title("Clusters: Hour vs Resolve Time")
plt.tight_layout()
plt.show()
